In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
stations = sorted(glob.glob('raw_data/*_stations*'))
rain_data = sorted(glob.glob('raw_data/*_precipitation*'))

dates = np.array([i.split('/')[-1].split('_')[0] for i in stations])

*TO-DO*
- [x] Obtain a master reference for all stations that are present within the dataset
- [x] Function to arrange rainfall must
    1. Be able to have data for all dates present in the raw data folder in one dataframe
    2. Be able to create a dataframe that isn't dependant on having the same stations across all source data

In [3]:
def concat_stations(station_ids):
    n = station_ids.size
    for i in range(0, n-1):
        df = pd.concat([station_ids[i], station_ids[i+1]], ignore_index=True)

    df.drop_duplicates(subset = 'device_id', ignore_index = True, inplace = True)
    return df

def collate_data(all_stations, sum_preci, dates):
    row = dates.size
    column_id = np.array(all_stations['device_id'])

    df = pd.DataFrame(np.zeros((row, column_id.size)), columns = column_id, index = dates)

    for i, series in enumerate(sum_preci):
        cols = series.columns
        vals = series.to_numpy()
        df.loc[dates[i], cols] = vals[0]

    return df 

In [4]:
station_ids = np.zeros(len(stations), dtype = object)
sum_rain = np.zeros(len(stations), dtype = object)

for i in range(len(stations)):
    station = pd.read_csv(stations[i])
    station = station.drop(['Unnamed: 0', 'id'], axis = 1)
    station = station.astype({'longitude':'float','latitude':'float'})
    
    rain = pd.read_csv(rain_data[i])
    times = rain['Time'].to_numpy()
    rain = rain.drop(['Time'], axis = 1)
    rain = rain.astype(float)
    
    station_ids[i] = station
    sum_rain[i] = pd.DataFrame(rain.sum()).T

all_stations = concat_stations(station_ids)
all_rain = collate_data(all_stations, sum_rain, dates)

In [5]:
all_rain.to_csv('./collated_daily_data.csv')
all_stations.to_csv('./all_stations.csv')

In [6]:
all_rain

,S77,S109,S90,S114,S50,S107,S215,S118,S120,S33,...,S89,S115,S82,S24,S69,S08,S116,S104,S100,S36
2022-08-12,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,9.8,0.0
2022-08-13,3.2,17.0,3.4,15.4,13.4,1.8,11.977,0.6,0.4,14.2,...,26.4,3.4,24.0,32.4,2.8,4.6,1.0,2.4,2.2,0.0
2022-08-14,0.0,9.8,0.0,18.2,0.4,0.0,0.000,0.0,0.0,0.0,...,0.2,0.2,1.6,0.0,37.0,7.8,0.0,30.4,31.2,0.0
2022-08-15,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.2,4.4,0.0,0.0
